#### This notebook demonstrates the use of Reweighing pre-processing, Adversarial Debiasing in-processing and Reject Option Classification (ROC) post-processing algorithms for bias mitigation.
- Load imports
- Dataset
    * Load Adult, COMPAS, or German dataset and set privileged and unprivileged groups
    * Divide the dataset into training, validation, and testing partitions
    * Show dataset properties
- Pre-processing: Reweighing.
    * Show difference in mean outcomes for original training data
    * Assign weights with reweighing
    * Show difference in mean outcomes for transformed training data
- In-processing: Adversarial Debiasing.
    * Train model without debiasing, predict, and show metrics
    * Train model with debiasing, predict, and show metrics
- Post-processing: Reject Option Classification (ROC).
    * Show metrics for test set from Adversarial Debiasing without debiasing
    * Fit ROC model
    * Transform labels and show metrics

In [1]:
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
import tensorflow as tf
from warnings import warn 

# Avoid deprecation warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas

from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification

from common_utils import compute_metrics

from IPython.display import Markdown, display
from ipywidgets import interactive, FloatSlider

#### Load dataset and specify options

In [2]:
## import dataset
dataset_used = "german" # "adult", "german", "compas"
protected_attribute_used = 1 # 1, 2

if dataset_used == "adult":
#     dataset_orig = AdultDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
        dataset_orig = load_preproc_data_adult(['sex'])
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]
        dataset_orig = load_preproc_data_adult(['race'])
    
elif dataset_used == "german":
#     dataset_orig = GermanDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
        dataset_orig = load_preproc_data_german(['sex'])
    else:
        privileged_groups = [{'age': 1}]
        unprivileged_groups = [{'age': 0}]
        dataset_orig = load_preproc_data_german(['age'])
    
elif dataset_used == "compas":
#     dataset_orig = CompasDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 0}]
        unprivileged_groups = [{'sex': 1}]
        dataset_orig = load_preproc_data_compas(['sex'])
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]  
        dataset_orig = load_preproc_data_compas(['race'])

#### Split into train, test and validation

In [3]:
# Get the dataset and split into train and test
dataset_orig_train, dataset_orig_vt = dataset_orig.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

#### Clean up training data and display properties of the data

In [4]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(700, 11)


#### Favorable and unfavorable labels

1.0 2.0


#### Protected attribute names

['sex']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['age', 'sex', 'credit_history=Delay', 'credit_history=None/Paid', 'credit_history=Other', 'savings=500+', 'savings=<500', 'savings=Unknown/None', 'employment=1-4 years', 'employment=4+ years', 'employment=Unemployed']


## Pre-processing: Reweighing

#### Metric for original training data

In [5]:
# 1. Crear un objeto de la clase BinaryLabelDatasetMetric cuyo constructor recibe por parámetros el dataset de entrenamiento, 
# el array de tuplas de grupos desprivilegiados y el array de tuplas de grupos privilegiados. 
# 2. Llamar a su método mean_difference() e imprimir el valor devuelto (un float)

#### Reweighing

In [6]:
# 1. Crear un objeto de la clase Reweighing(grupos desprivilegiados, gurpos privilegiados)
# 2. Llamar a su método fit(dataset) con el dataset de entrenamiento
# 3. Llamar a su método transform(dataset) con el dataset de entrenamiento que devuelve el dataset transformado

#### Metric for reweighted training data

In [7]:
# 1. Imprimir la media de diferencias del dataset transformado

## In-processing: Adversarial Debiasing

### Without debiasing

#### Train without debiasing

In [8]:
# Learn parameters with debias set to False
# 1. Iniciar una sesión de tensorflow con tf.Session() 
# 2. Crear un objeto de la clase AdversarialDebiasing(grupo privilegiado, grupo desprivilegiado, 'plain_classifier', 
# booleano de si queremos que haya un adversario (en este caso False), la sesión de tensorflow)

In [9]:
# 1. Llamar a su método fit(dataset) con el dataset de entrenamiento

#### Show metrics

In [10]:
# Apply the plain model to test data
# 1. Llamar a su método predict(dataset) que devuelve un dataset con las labels predichas para el dataset de entrenamiento, 
# validación y testeo. Guardar cada uno en 3 datasets distintos (entre sí y respecto a los dataset de entrada)!

In [11]:
# Metrics for the dataset from plain model (without debiasing)
# 1. Imprimir la media de diferencias del dataset de entrenamiento y el dataset de test

In [12]:
# Accuracy
# 1. Crear un objeto de la clase ClassificationMetric(dataset no predicho, dataset predicho, grupos desprivilegiados, 
# grupos privilegiados) con el dataset de test
# 2. Llamar a su método accuracy() e imprimir el valor devuelto (un float)


# Other metrics
# 1. Llamar a la función compute_metrics(dataset no predicho, dataset predicho, grupos desprivilegiados, grupos privilegiados)

In [13]:
# Compute scores for ROC
# 1. Llamar al método del objeto de la clase AdversarialDebiasing predict_proba(dataset no predicho) 
# con el dataset de validación y el de test
# 2. Transformar el valor devuelto en una columna con reshape y guardar los valores en el atributo scores de los 
# datasets predichos de validación y de test respectivamente

### With debiasing

#### Train with debiasing

In [14]:
# 1. Cerrar la sesión de tf con su método close()
# 2. Llamar a la función de tf reset_default_graph()
# 3. Abrir una nueva sesión de tensorflow

In [15]:
# Learn parameters with debias set to True
# 1. Volver a crear un objeto de la clase AdversarialDebiasing con debias=True

In [16]:
# 1. Entrenar el modelo sobre el dataset de entrenamiento

#### Show metrics

In [17]:
# Apply the plain model to test data
# 1. Predecir el dataset de entrenamiento y de test

In [18]:
# Metrics for the dataset from plain model (without debiasing)
# 1. Mostrar la media de las diferencias del dataset de entrenamiento y de test

In [19]:
# Accuracy
# 1. Mostrar la precisión del dataset test


# Other metrics
# 1. Mostrar el resto de métricas del dataset test

## Post-processing: Reject Option Classification

#### Show metrics for Test Set

In [20]:
# Metrics for the test set
# Accuracy 
# 1. Mostrar la precisión del dataset test sin debias

# Other metrics
# 1. Mostrar el resto de métricas del dataset test sin debias

#### Estimate optimal parameters for the ROC method

In [21]:
# Metric used (should be one of allowed_metrics)
metric_name = "Statistical parity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
np.random.seed(1)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

In [22]:
# 1. Crear un objeto de la clase RejectOptionClassification(grupos desprivilegiados, gurpos privilegiados, 
# límite inferior de prueba, limite superior de prueba, número de thresholds, número de márgenes, métrica a optimizar, 
# límite superior de la métrica, límite inferior de la métrica)
# 2. Llamar a su método fit(dataset no predicho, dataset predicho) con el dataset de validación. Devuelve el ROC entrenado

In [23]:
# 1. Imprimir el classification_threshold y el ROC_margin del ROC entrenado

#### Show predictions from Test Set with ROC

In [24]:
# Metrics for the transformed test set
# 1. Predecir con predict(dataset predicho) las nuevas predicciones

# Accuracy 
# 1. Mostrar la precisión del dataset transformado test sin debias

# Other metrics
# 1. Mostrar el resto de métricas del dataset transformado test sin debias

References:

F. Kamiran, and T. Claders,"Data preprocessing techniques for classification without discrimination",
Knowledge and Information Systems, 33(1):1–33, 2012. 

B. H. Zhang, B. Lemoine, and M. Mitchell, "Mitigating UnwantedBiases with Adversarial Learning",
AAAI/ACM Conference on Artificial Intelligence, Ethics, and Society, 2018.

F. Kamiran, A. Karim, and X. Zhang,  "Decision theory for discrimination-aware classification",
In IEEE International Conference on Data Mining, pp. 924–929, 2012.